In [6]:
import math
import random
import torch
import numpy as np

class NeighborhoodSampling:
    """
        checkboard:   0  1  2
                      3  4  5
                      6  7  8
        context boxes: [0, 1, 2, 3, 5, 6, 7, 8]
        candidate_groups: 2 ** 8 = 256
        box: tensor
    """
    def __init__(self,
                 max_groups=4,
                 max_permutations=4,
                 alpha=3.0,
                 cut_off_thr=0.5,
                 base_probability=0.5,
                 interval=0.0,
                 **kwargs):
        
        self.max_groups = max_groups
        self.max_permutations = max_permutations
        self.alpha = alpha
        self.cut_off_thr = cut_off_thr
        self.base_probability = base_probability
        self.interval = interval
        
        
        box_ids = []
        left_right_up_downs = []
        box_templates = []
        for i in range(3):
            h_interval = (float(i) - 1.0) * self.interval
            for j in range(3):
                w_interval = (float(j) - 1.0) * self.interval
                box = [float(j) + w_interval, float(i) + h_interval,
                       float(j+1) + w_interval, float(i+1) + h_interval]
                box_templates.append(box)
        self.box_templates = np.array(box_templates, dtype=np.float32)
        self.binary_mask_template = 10 ** np.arange(9, dtype=np.float32)


        for l in range(2):       # left: -1
            for r in range(2):   # right +1
                for u in range(2):    # up -3
                    for d in range(2):  # down  +3
                        left_right_up_downs.append([l, r, u, d])
                        box_ids.append(list({4-l-3*u, 4-3*u, 4+r-3*u,
                                             4-l,     4,     4+r,
                                             4-l+3*d, 4+3*d, 4+r+3*d}))
        self.box_ids = box_ids
        self.left_right_up_downs = np.array(left_right_up_downs, dtype=np.float32)
        self.context_box_ids = [0, 1, 2, 3, 5, 6, 7, 8]
        
        
        print(f'box_templates: {self.box_templates}')
        print(f'box_ids: {self.box_ids}')
        print(f'binary_mask_template: {self.binary_mask_template}')
        print(f'left_right_up_downs: {self.left_right_up_downs}')
        


In [9]:
nhs = NeighborhoodSampling()


box_templates: [[0. 0. 1. 1.]
 [1. 0. 2. 1.]
 [2. 0. 3. 1.]
 [0. 1. 1. 2.]
 [1. 1. 2. 2.]
 [2. 1. 3. 2.]
 [0. 2. 1. 3.]
 [1. 2. 2. 3.]
 [2. 2. 3. 3.]]
box_ids: [[4], [4, 7], [1, 4], [1, 4, 7], [4, 5], [8, 4, 5, 7], [1, 2, 4, 5], [1, 2, 4, 5, 7, 8], [3, 4], [3, 4, 6, 7], [0, 1, 3, 4], [0, 1, 3, 4, 6, 7], [3, 4, 5], [3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5, 6, 7, 8]]
binary_mask_template: [1.e+00 1.e+01 1.e+02 1.e+03 1.e+04 1.e+05 1.e+06 1.e+07 1.e+08]
left_right_up_downs: [[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 1.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 1.]
 [1. 0. 1. 0.]
 [1. 0. 1. 1.]
 [1. 1. 0. 0.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]
 [1. 1. 1. 1.]]


In [ ]:
<InstanceData(

    META INFORMATION
    pad_shape: (736, 992)
    img_path: '/work/ovdet/data/coco/train2017/000000429593.jpg'
    ori_shape: (480, 640)
    img_shape: (736, 981)
    flip: False
    img_id: 429593
    flip_direction: None
    scale_factor: (1.5328125, 1.5333333333333334)
    batch_input_shape: (736, 992)

    DATA FIELDS
    scores: tensor([1.0000, 1.0000, 0.9995, 0.9995, 0.9990, 0.9985, 0.9985, 0.9946, 0.9941,
                0.9893, 0.9854, 0.9790, 0.9536, 0.9097], device='cuda:0')
    bboxes: tensor([[914.1723, 540.0206, 975.8277, 673.9794],
                [664.4435, 303.4160, 881.5052, 503.6693],
                [245.9763, 495.5276, 533.5237, 567.4724],
                [  5.8639, 415.7931, 118.1361, 536.2069],
                [ 32.9593,  98.2998, 981.0000, 736.0000],
                [666.2534, 213.6198, 825.7466, 321.3802],
                [ 26.4330,   7.1967, 280.8170, 293.5533],
                [261.0216, 393.5546, 400.4784, 470.4454],
                [114.1597, 334.7249, 219.0903, 459.7751],
                [550.5017, 680.2295, 717.4983, 723.7705],
                [126.7657, 504.3645, 265.2343, 568.6355],
                [568.2763, 343.0470, 623.7237, 487.9530],
                [872.4855,  17.8971, 963.5145, 107.1654],
                [262.6717,  86.1948, 339.3283, 181.8052]], device='cuda:0')

In [10]:
import torch
scaled_bbox  = torch.tensor([[ 488.2089,  811.9690,  679.3351, 1092.7760]])

scaled_bbox[[[0,0]]] /= 2

print(scaled_bbox)

IndexError: index 2 is out of bounds for dimension 0 with size 1

In [ ]:
[[0.6035, 0.6260, 0.6260, 0.6060, 0.6060, 0.4102, 0.4102, 0.4148, 0.3523, 0.3523],
 [0.5972, 0.6421,   -inf, 0.6016, 0.6016, 0.3845, 0.3845, 0.3752, 0.3567, 0.3567],
 [0.5688,   -inf, 0.6313, 0.6069, 0.6069, 0.3840, 0.3840, 0.3804, 0.3391, 0.3391],
 [0.5327, 0.5894, 0.5894, 0.5869,   -inf, 0.3784, 0.3784, 0.3584, 0.3311, 0.3311],
 [0.4973, 0.5469, 0.5469,   -inf, 0.5640, 0.3660, 0.3660, 0.3372, 0.3108, 0.3108],
 [0.3708, 0.3650, 0.3650, 0.4307, 0.4307, 0.5928,   -inf, 0.5752, 0.4429, 0.4429],
 [0.3381, 0.3369, 0.3369, 0.3904, 0.3904,   -inf, 0.5811, 0.5713, 0.4309, 0.4309],
 [0.3977, 0.4116, 0.4116, 0.4099, 0.4099, 0.5723, 0.5723, 0.6152, 0.4602, 0.4602],
 [0.3281, 0.3159, 0.3159, 0.3108, 0.3108, 0.5049, 0.5049, 0.5327, 0.5234,   -inf],
 [0.3237, 0.3167, 0.3167, 0.3247, 0.3247, 0.4253, 0.4253, 0.4412,   -inf, 0.4844]]